## カウントベースによる手法：TF-IDF


In [2]:
%pip install janome

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter

#日本語テキストを分かち書きするクラス
class WordDividor:
  def extract_words(self, text):
    filters = [ POSKeepFilter( ['名詞'] )]
    analyzer = Analyzer(token_filters = filters)
    return [token.surface for token in analyzer.analyze(text)]

#対象文書
docs = [
  '吾輩は猫である．名前はまだない',
  '一匹の立派な犬や兎や一匹の子猿や一匹の猫などを飼った',
  '兎やキツネの他に，イタチの足あと，ネズミの足あと，猫の足あと，みんなちがう'  
]

tokenizer = WordDividor() #tokenizerの生成

#countエンコーディング (binary=Trueの場合はOne-Hotエンコーディング)
vectorizer = CountVectorizer( binary=False, tokenizer=tokenizer.extract_words )
bow = vectorizer.fit_transform( docs ) 
#print(vectorizer.vocabulary_)
print( sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]) )
print( bow.toarray() )

print( ' ---- ')

#tfidf
tfidf = TfidfVectorizer(tokenizer=tokenizer.extract_words, smooth_idf=False)
tfidf_vector = tfidf.fit_transform(docs)
#print(tfidf.vocabulary_)
print( sorted(tfidf.vocabulary_.items(), key=lambda x:x[1]) )
print( tfidf_vector.toarray().round(2) )


[('あと', 0), ('みんな', 1), ('イタチ', 2), ('キツネ', 3), ('ネズミ', 4), ('一', 5), ('他', 6), ('兎', 7), ('匹', 8), ('名前', 9), ('吾輩', 10), ('犬', 11), ('猫', 12), ('猿', 13), ('立派', 14), ('足', 15)]
[[0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]
 [0 0 0 0 0 3 0 1 3 0 0 1 1 1 1 0]
 [3 1 1 1 1 0 1 1 0 0 0 0 1 0 0 3]]
 ---- 
[('あと', 0), ('みんな', 1), ('イタチ', 2), ('キツネ', 3), ('ネズミ', 4), ('一', 5), ('他', 6), ('兎', 7), ('匹', 8), ('名前', 9), ('吾輩', 10), ('犬', 11), ('猫', 12), ('猿', 13), ('立派', 14), ('足', 15)]
[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.67 0.67 0.   0.32 0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.64 0.   0.14 0.64 0.   0.   0.21 0.1  0.21
  0.21 0.  ]
 [0.62 0.21 0.21 0.21 0.21 0.   0.21 0.14 0.   0.   0.   0.   0.1  0.
  0.   0.62]]


## LDA（参考）


In [4]:
#必要なライブラリのインストール
!pip install janome beautifulsoup4  

In [5]:
#データのダウンロード
!wget https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

--2022-07-03 23:45:14--  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz
s3.amazonaws.com (s3.amazonaws.com) をDNSに問いあわせています... 52.216.114.77
s3.amazonaws.com (s3.amazonaws.com)|52.216.114.77|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 94688992 (90M) [application/x-gzip]
`amazon_reviews_multilingual_JP_v1_00.tsv.gz.1' に保存中

amazon_reviews_mult 100%[===================>]  90.30M  7.31MB/s 時間 12s        

2022-07-03 23:45:27 (7.56 MB/s) - `amazon_reviews_multilingual_JP_v1_00.tsv.gz.1' へ保存完了 [94688992/94688992]



In [6]:
#データの読み込みと前処理
import pandas as pd
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter
from sklearn.feature_extraction.text import CountVectorizer

#データの読み込みとフィルタリング
def load_dataset(filename, n=1000, random_state = 42):
    df = pd.read_csv(filename, sep='\t') #データ読み込み
    df = df.sample(frac=1, random_state=random_state)  # shuffle
    grouped = df.groupby('star_rating') #各ratingでn件のレビューを得る
    df = grouped.head(n=n)
    return df.review_body.values, df.star_rating.values

class Preprocessor:
  def __init__(self):
    self.filters = [ POSKeepFilter( ['名詞','形容詞','副詞'] )]
    self.analyzer = Analyzer(token_filters = self.filters)

  def extract_words(self, text):
    soup = BeautifulSoup(text, 'html.parser') #htmlの除去
    text = soup.get_text(strip=True)
    return [token.surface for token in self.analyzer.analyze(text)]

url = 'amazon_reviews_multilingual_JP_v1_00.tsv.gz'
reviews, ratings = load_dataset(url, n=1000) #各クラスごとにn=1000件を選択

tokenizer = Preprocessor() #tokenizerの生成
#countエンコーディング (df >= 0.3なる単語は排除，頻度上位100件)
counter = CountVectorizer( binary=False, tokenizer=tokenizer.extract_words, max_features=100, max_df=0.3 )
review_bow = counter.fit_transform( reviews ) #bow表現へ



EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 10  #tトピック数の設定
lda_model = LatentDirichletAllocation( n_components = n_topics)
lda_model.fit( review_bow ) #LDAの実行
review_vec = lda_model.transform(review_bow) #各文書に対するベクトル
#print( review_vec.shape )

In [ ]:
n_of_words = 10
feature_names = counter.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_) :
  print("Topic %d:" %(topic_idx+1) ,end=" ")
  ws = [ feature_names[i] for i in topic.argsort()[:-n_of_words-1 : -1]]
  print(" ".join(ws))

print()
target_topic = 3
for movie_idx in (review_vec[:,target_topic].argsort()[::-1])[:3] :
  print(reviews[movie_idx])
  print()

## word2vecベクトルの利用：swem
  

In [ ]:
# 前回ダウンロードしています．残っていれば，改めてDLする必要はありません
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz .
# !wget https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.entity_vectors.100d.txt.bz2 .


In [ ]:
#必要なライブラリとswemのインストール
!pip install mecab-python3==0.996.2 #形態素解析器 Mecabのインストール
!git clone https://github.com/yagays/swem.git

In [ ]:
import os
import sys
sys.path.append("swem")

from gensim.models import KeyedVectors
from swem import MeCabTokenizer
from swem import SWEM

#w2v_path = "/path/to/word_embedding.bin" #word2vecのファイルを指定する
w2v_path = "jawiki.entity_vectors.100d.txt.bz2" #自身の環境に合わせて，パスを変更

w2v = KeyedVectors.load_word2vec_format(w2v_path, binary=False) #読み込みに時間がかかります
tokenizer = MeCabTokenizer("-O wakati")

swem = SWEM(w2v, tokenizer)

In [ ]:
import numpy as np
docs = [
  '吾輩は猫である．名前はまだない',
  '一匹の立派な犬や兎や一匹の子猿や一匹の猫などを飼った',
  '兎やキツネの他に，イタチの足あと，ネズミの足あと，猫の足あと，みんなちがう'  
]
vec_avgs = [ swem.average_pooling(d) for d in docs ]
vec_maxs = [ swem.max_pooling(d) for d in docs ]
vec_concats = [ swem.concat_average_max_pooling(d) for d in docs]
vec_hiers = [ swem.hierarchical_pooling(d, n=3) for d in docs]


In [ ]:
def cos_similarity(x, y, eps=1e-8):
  nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
  ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
  return np.dot(nx, ny)

for i in range(len(docs)):
  cos_avg = cos_similarity(vec_avgs[i], vec_avgs[(i+1)%len(docs)] )
  cos_max = cos_similarity(vec_maxs[i], vec_maxs[(i+1)%len(docs)] )
  cos_con = cos_similarity(vec_concats[i], vec_concats[(i+1)%len(docs)] )
  cos_hie = cos_similarity(vec_hiers[i], vec_hiers[(i+1)%len(docs)] )
  print(i, (i+1), cos_avg, cos_max, cos_con, cos_hie)



## 推論ベースの手法:doc2vec (参考)
[日本語WIKIPEDIAで学習したDOC2VECモデル](https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/)より  
モデルが大きいです．無理に動かす必要はありません


In [ ]:
#dbowモデル(5.48GB)を獲得
!wget https://www.dropbox.com/s/j75s0eq4eeuyt5n/jawiki.doc2vec.dbow300d.tar.bz2 .  #dl
!tar xfvj jawiki.doc2vec.dbow300d.tar.bz2 #解凍

In [ ]:
import MeCab
def tokenize(text):
    wakati = MeCab.Tagger("-O wakati")
    wakati.parse("")
    return wakati.parse(text).strip().split()

from gensim.models.doc2vec import Doc2Vec
model_path = "jawiki.doc2vec.dbow300d.model" #自身の環境に合わせて，パスを変更
model = Doc2Vec.load(model_path) #モデルの読み込み 時間がかかります

In [ ]:
text = """
吾輩は猫である．名前はまだない．
"""
vec = model.infer_vector(tokenize(text)) #文を分かち書きし，ベクトルを推定する（文書ベクトルの獲得）
model.docvecs.most_similar([vec]) #得られたベクトルを使って類似センテンスを獲得

## 推論ベースの手法：BERT
モデルのロードと文書ベクトルの獲得

In [ ]:
!pip install transformers fugashi ipadic

In [ ]:
from transformers import BertJapaneseTokenizer, BertModel
import pandas as pd
import numpy as np
import torch

#初回実行時にモデルをダウンロードします
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

In [ ]:
# 文書ベクトル獲得メソッド
def get_bert_vectors( model, sentences) :
  encoded_data = tokenizer.batch_encode_plus(sentences, padding='longest') 
  input_ids = torch.tensor(encoded_data["input_ids"])

  with torch.no_grad(): # 勾配計算を行わない
    outputs = model( input_ids )
  sentence_vecs = outputs[0][:,0,:] #最終層の重みを獲得
  sentence_vecs = sentence_vecs.to('cpu').detach().numpy().copy() #numpy配列に変換
  return sentence_vecs

In [ ]:
#ベクトルの獲得例
sentences = [ 
  '吾輩は猫である．名前はまだない',
  '一匹の立派な犬や兎や一匹の子猿や一匹の猫などを飼った',
  '兎やキツネの他に，イタチの足あと，ネズミの足あと，猫の足あと，みんなちがう'
]

result = get_bert_vectors(model, sentences)

print( np.shape( result ) ) #各文が768次元ベクトルに
print( result )